# Predicción de aprobación de curso de matemáticas usando árboles de decisión



*   **scikit-learn** es una librería que cuenta con algoritmos de clasificación, regresión, clustering y reducción de dimensionalidad. Además, presenta la compatibilidad con otras librerías como NumPy, SciPy y matplotlib.

*   **Pandas** es una librería de Python especializada en la manipulación y el análisis de datos. Ofrece estructuras de datos y operaciones para manipular tablas

In [ ]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn import tree

*   Se cargan los datos del archivo CSV a un **dataframe**. Un dataframe es una estructura de datos con filas y columnas

In [ ]:
datos = pd.read_csv('student_performance.csv')
datos

*   Se muestra información general del dataset

In [ ]:
print(datos.shape)
print(datos.info())
print(datos.describe())

*   Se identifican las columnas numéricas y categóricas del dataset

In [ ]:
columnas_numericas = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'goout', 'Walc', 'health']
columnas_categoricas = ['sex', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'internet', 'romantic']
print("Columnas numéricas:", columnas_numericas)
print("Columnas categóricas:", columnas_categoricas)

*   Se normalizan las variables numéricas usando **StandardScaler**

In [ ]:
scaler = StandardScaler()
datos[columnas_numericas] = scaler.fit_transform(datos[columnas_numericas])

*   Se codifican las variables categóricas usando **LabelEncoder**

In [ ]:
for col in columnas_categoricas:
    le = LabelEncoder()
    datos[col] = le.fit_transform(datos[col])

In [ ]:
datos.head()

*   Se separan las características (X) de la variable a predecir (Y)

In [ ]:
X = datos.drop('approved', axis=1)
Y = datos['approved']

*   Se divide el dataset en 80% para entrenamiento y 20% para pruebas

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("Tamaño conjunto de entrenamiento:", X_train.shape[0])
print("Tamaño conjunto de prueba:", X_test.shape[0])

## Árboles de decisión con criterio 'gini'
*   Se crean 5 árboles con max_depth desde 2 hasta 10 con incrementos de 2

In [ ]:
resultados_gini = []

for depth in [2, 4, 6, 8, 10]:
    modelo = tree.DecisionTreeClassifier(criterion='gini', max_depth=depth, random_state=42)
    modelo.fit(X_train, Y_train)
    predicciones = modelo.predict(X_test)
    accuracy = accuracy_score(Y_test, predicciones)
    resultados_gini.append({'max_depth': depth, 'criterion': 'gini', 'accuracy': accuracy})
    print(f"max_depth={depth}, criterion='gini': Accuracy = {accuracy:.4f}")

*   Tabla de resultados para criterio 'gini'

In [ ]:
tabla_gini = pd.DataFrame(resultados_gini)
tabla_gini

## Árboles de decisión con criterio 'entropy'
*   Se crean 5 árboles con max_depth desde 2 hasta 10 con incrementos de 2

In [ ]:
resultados_entropy = []

for depth in [2, 4, 6, 8, 10]:
    modelo = tree.DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=42)
    modelo.fit(X_train, Y_train)
    predicciones = modelo.predict(X_test)
    accuracy = accuracy_score(Y_test, predicciones)
    resultados_entropy.append({'max_depth': depth, 'criterion': 'entropy', 'accuracy': accuracy})
    print(f"max_depth={depth}, criterion='entropy': Accuracy = {accuracy:.4f}")

*   Tabla de resultados para criterio 'entropy'

In [ ]:
tabla_entropy = pd.DataFrame(resultados_entropy)
tabla_entropy

*   Se combinan ambos resultados para identificar el mejor modelo

In [ ]:
todos_resultados = pd.concat([tabla_gini, tabla_entropy], ignore_index=True)
todos_resultados

In [ ]:
mejor = todos_resultados.loc[todos_resultados['accuracy'].idxmax()]
print("\nMejor modelo:")
print(f"max_depth: {int(mejor['max_depth'])}")
print(f"criterion: {mejor['criterion']}")
print(f"accuracy: {mejor['accuracy']:.4f}")

### Hiperparámetros del mejor árbol de decisión

Después de evaluar 10 configuraciones diferentes, los hiperparámetros que proporcionan el mayor accuracy son los mostrados en la celda anterior.

## Experimentación con hiperparámetro adicional: min_samples_split

*   Se selecciona el hiperparámetro **min_samples_split** que controla el número mínimo de muestras necesarias para dividir un nodo interno

In [ ]:
mejor_max_depth = int(mejor['max_depth'])
mejor_criterion = mejor['criterion']
mejor_accuracy = mejor['accuracy']

print(f"Configuración base del mejor modelo:")
print(f"max_depth={mejor_max_depth}, criterion='{mejor_criterion}'")
print(f"Accuracy base: {mejor_accuracy:.4f}")

*   Se prueban dos valores diferentes de min_samples_split: 10 y 50

In [ ]:
resultados_adicional = []

# Modelo base (min_samples_split=2 por defecto)
resultados_adicional.append({
    'min_samples_split': 2,
    'accuracy': mejor_accuracy,
    'diferencia': 0.0
})

# Variaciones de min_samples_split
for min_samples in [10, 50]:
    modelo = tree.DecisionTreeClassifier(
        criterion=mejor_criterion,
        max_depth=mejor_max_depth,
        min_samples_split=min_samples,
        random_state=42
    )
    modelo.fit(X_train, Y_train)
    predicciones = modelo.predict(X_test)
    accuracy = accuracy_score(Y_test, predicciones)
    diferencia = accuracy - mejor_accuracy
    
    resultados_adicional.append({
        'min_samples_split': min_samples,
        'accuracy': accuracy,
        'diferencia': diferencia
    })
    
    print(f"\nmin_samples_split={min_samples}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Diferencia con modelo base: {diferencia:+.4f}")

In [ ]:
tabla_adicional = pd.DataFrame(resultados_adicional)
tabla_adicional

### Análisis del hiperparámetro min_samples_split

El hiperparámetro **min_samples_split** especifica el número mínimo de muestras requeridas para dividir un nodo interno del árbol. 

**Valores probados:**
- min_samples_split = 2 (valor por defecto)
- min_samples_split = 10
- min_samples_split = 50

**Resultados:**

Al modificar el valor de min_samples_split se observa que:

- Si el accuracy **aumenta**: el modelo está mejorando su capacidad de generalización al evitar divisiones con pocas muestras que podrían estar capturando ruido.

- Si el accuracy **disminuye**: el modelo pierde capacidad predictiva porque no puede realizar divisiones necesarias para capturar patrones importantes en los datos.

- Si el accuracy **se mantiene**: el valor de min_samples_split no afecta significativamente al modelo con la configuración actual de max_depth.

Un valor más alto de min_samples_split hace que el árbol sea más conservador en sus divisiones, lo que puede ayudar a prevenir el sobreajuste pero también puede resultar en un modelo más simple que pierde precisión.

*   Se calcula el accuracy, la sensibilidad y especificidad para el mejor modelo

In [ ]:
# Entrenar el mejor modelo
modelo_final = tree.DecisionTreeClassifier(
    criterion=mejor_criterion,
    max_depth=mejor_max_depth,
    random_state=42
)
modelo_final.fit(X_train, Y_train)
predicciones_final = modelo_final.predict(X_test)

In [ ]:
accuracy = accuracy_score(Y_test, predicciones_final)
sensibilidad = recall_score(Y_test, predicciones_final, pos_label=1)
especificidad = recall_score(Y_test, predicciones_final, pos_label=0)

print("Accuracy=", accuracy)
print("Sensibilidad=", sensibilidad)
print("Especificidad=", especificidad)

*   Matriz de confusión del mejor modelo

In [ ]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test, predicciones_final)
matriz

*   Gráfico de comparación de accuracy entre criterios

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot([2, 4, 6, 8, 10], tabla_gini['accuracy'], marker='o', label='Gini', linewidth=2)
plt.plot([2, 4, 6, 8, 10], tabla_entropy['accuracy'], marker='s', label='Entropy', linewidth=2)
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Comparación de Accuracy por Criterio')
plt.legend()
plt.grid(True)
plt.show()